# **Закрепление знаний**

Этот набор данных создан для того, чтобы на основе определённых диагностических измерений предсказать, есть ли у пациента диабет. 

На выбор этих экземпляров из более крупной базы данных было наложено несколько ограничений. В частности, все пациенты здесь — женщины не моложе 21 года индейского происхождения Пима.

In [10]:
import pandas as pd
from IPython.display import display
import numpy as np
diabetes = pd.read_csv('data/diabetes_data.csv')
display(diabetes.shape[0])
diabetes.head(15)

778

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome,Gender
0,6,98,58,33,190,34.0,0.430,43,0,Female
1,2,112,75,32,0,35.7,0.148,21,0,Female
2,2,108,64,0,0,30.8,0.158,21,0,Female
3,8,107,80,0,0,24.6,0.856,34,0,Female
4,7,136,90,0,0,29.9,0.210,50,0,Female
5,6,103,72,32,190,37.7,0.324,55,0,Female
6,1,71,48,18,76,20.4,0.323,22,0,Female
7,0,117,0,0,0,33.8,0.932,44,0,Female
8,4,154,72,29,126,31.3,0.338,37,0,Female
9,5,147,78,0,0,33.7,0.218,65,0,Female


In [12]:
# Начнём с поиска дубликатов в данных. Найдите все повторяющиеся строки в данных и удалите их.
# Для поиска используйте все признаки в данных. Сколько записей осталось в данных?

dupl_columns = list(diabetes.columns)

mask = diabetes.duplicated(subset=dupl_columns) # Создадим маску дубликатов с помощью метода duplicated() и произведём фильтрацию
diabetes_duplicates = diabetes[mask]
print(f'Число найденных дубликатов: {diabetes_duplicates.shape[0]}')

diabetes = diabetes.drop_duplicates(subset=dupl_columns)
print(f'Результирующее число записей: {diabetes_dedupped.shape[0]}')

Число найденных дубликатов: 10
Результирующее число записей: 768


In [15]:
# Далее найдите все неинформативные признаки в данных и избавьтесь от них.
# В качестве порога информативности возьмите 0.99: удалите все признаки,
# для которых 99 % значений повторяются или 99 % записей уникальны.
# В ответ запишите имена признаков, которые вы нашли (без кавычек).

#список неинформативных признаков
low_information_cols = []

#цикл по всем столбцам
for col in diabetes.columns:
    #наибольшая относительная частота в признаке
    top_freq = diabetes[col].value_counts(normalize=True).max()
    #доля уникальных значений от размера признака
    nunique_ratio = diabetes[col].nunique() / diabetes[col].count()
    # сравниваем наибольшую частоту с порогом
    if top_freq > 0.95:
        low_information_cols.append(col)
        print(f'{col}: {round(top_freq*100, 2)}% одинаковых значений')
    # сравниваем долю уникальных значений с порогом
    if nunique_ratio > 0.95:
        low_information_cols.append(col)
        print(f'{col}: {round(nunique_ratio*100, 2)}% уникальных значений')
        
diabetes = diabetes.drop(low_information_cols, axis=1)
display(diabetes.shape[1])

9

In [ ]:
# Замените все записи, равные 0, в столбцах Glucose, BloodPressure, SkinThickness,
# Insulin и BMI на символ пропуска. Его вы можете взять из библиотеки numpy: np.nan.

for column in diabetes['Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI']:
    for row in column:
        if row==0:
            row=np.nan